In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torchaudio
!pip install transformers
!pip install datasets
!pip install jiwer jiwer
!pip install pyarrow

     |████████████████████████████████| 1.9MB 11.2MB/s 
     |████████████████████████████████| 2.1MB 10.1MB/s 
     |████████████████████████████████| 3.3MB 40.9MB/s 
     |████████████████████████████████| 901kB 40.3MB/s 
     |████████████████████████████████| 225kB 10.6MB/s 
     |████████████████████████████████| 112kB 19.5MB/s 
     |████████████████████████████████| 245kB 18.2MB/s 
     |████████████████████████████████| 51kB 4.1MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149828 sha256=7484c52567970cb06acaa2e063f93c90683989b0f455a5f35101f70dda732c6b
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [ ]:
import json
import re
import numpy as np
import pandas as pd 
import torch
import torchaudio
import librosa
import IPython.display as ipd
import random
import datasets
import gc
import os
import pyarrow as pa
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor

In [ ]:
train_dataset = load_dataset("common_voice", "zh-CN", split="train")# +validation #cache_dir
test_dataset = load_dataset("common_voice", "zh-CN", split="test")

Dataset common_voice downloaded and prepared to /root/.cache/huggingface/datasets/common_voice/zh-CN/6.1.0/bb59ce0bb532485ab64b5d488a8dd2addc3104f694e06bcd2c272dc608bb1112. Subsequent calls will reuse this data.


Reusing dataset common_voice (/root/.cache/huggingface/datasets/common_voice/zh-CN/6.1.0/bb59ce0bb532485ab64b5d488a8dd2addc3104f694e06bcd2c272dc608bb1112)


In [ ]:
train = train_dataset.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
test = test_dataset.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [ ]:
def remove_special_characters(batch):
  punc = "！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏."
  batch["sentence"] = re.sub(r"[%s]+"%punc, "", batch["sentence"]) + " "
  return batch

In [ ]:
cv_train = train.map(remove_special_characters)
cv_test = test.map(remove_special_characters)

In [ ]:
cv_train = cv_train.filter(lambda x: os.path.getsize(x['path']) < 30000)
cv_test = cv_test.filter(lambda x: os.path.getsize(x['path']) < 30000)

### Build Vocabulary

In [ ]:
def extract_all_chars(batch):
  all_text = batch["sentence"]
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = cv_train.map(extract_all_chars, batched=True, batch_size=1, keep_in_memory=True, remove_columns=cv_train.column_names)
vocab_test = cv_test.map(extract_all_chars, batched=True, batch_size=1, keep_in_memory=True, remove_columns=cv_test.column_names)

In [ ]:
vocab_list = list(set("".join([sent[0] for sent in vocab_train["vocab"]])) | set("".join([sent[0] for sent in vocab_test["vocab"]])))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
#vocab_dict

In [ ]:
# replace whitespace to "|" to make it visible
vocab_dict["|"] = vocab_dict[" "] 
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

3188

In [ ]:
with open('/content/drive/MyDrive/585_Team_Folder/vocab_5.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

### Set up Tokenizer and Processor

In [ ]:
#vocab_6 - 30000
tokenizer = Wav2Vec2CTCTokenizer("/content/drive/MyDrive/585_Team_Folder/vocab_3w.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

### Convert and resample

In [ ]:
cv_train[0]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/6a9f2b2b774324aca897d6c3873fd8712e331ada5b3d83a3d34d7636777a6ba0/cv-corpus-6.1-2020-12-11/zh-CN/clips/common_voice_zh-CN_22063448.mp3',
 'sentence': '道光十四年出生 '}

In [ ]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["sentence"]
    return batch

In [ ]:
cv_train = cv_train.map(speech_file_to_array_fn, remove_columns=cv_train.column_names)
cv_test = cv_test.map(speech_file_to_array_fn, remove_columns=cv_test.column_names)

In [ ]:
def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]), 48_000, 16_000)
    batch["sampling_rate"] = 16_000
    return batch

In [ ]:
cv_train = cv_train.map(resample, num_proc=4)
cv_test = cv_test.map(resample, num_proc=4)

Process ForkPoolWorker-17:
Process ForkPoolWorker-16:
Process ForkPoolWorker-15:
Process ForkPoolWorker-14:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/dist-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/dist-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/dist-packages/multiprocess/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.7/dist-packages/multiprocess/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/multiprocess/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/d

In [ ]:
rand_int = random.randint(0, len(cv_train)-1)

ipd.Audio(data=np.asarray(cv_train[rand_int]["speech"]), autoplay=True, rate=16000)

In [ ]:
#rand_int = random.randint(0, len(cv_train)-1)

print("Target text:", cv_train[rand_int]["target_text"])
print("Input array shape:", np.asarray(cv_train[rand_int]["speech"]).shape)
print("Sampling rate:", cv_train[rand_int]["sampling_rate"])

Target text: 莫朗人口变化图示 
Input array shape: (57600,)
Sampling rate: 16000


### Prepare dataset for training

In [ ]:
def prepare_dataset(batch):
    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [ ]:
cv_train = cv_train.map(prepare_dataset, remove_columns=cv_train.column_names, batch_size=8, num_proc=4, batched=True)
cv_test = cv_test.map(prepare_dataset, remove_columns=cv_test.column_names, batch_size=8, num_proc=4, batched=True)

# Training

In [ ]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
cer_metric = datasets.load_metric("cer")

Couldn't find file locally at cer/cer.py, or remotely at https://raw.githubusercontent.com/huggingface/datasets/1.5.0/metrics/cer/cer.py.
The file was picked from the master branch on github instead at https://raw.githubusercontent.com/huggingface/datasets/master/metrics/cer/cer.py.


In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05, # important - can be optimized
    #mask_feature_prob should also be optimized 0.05 - 0.025. not too important.
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#model.to(device)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.freeze_feature_extractor()

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/drive/MyDrive/585_Team_Folder/model_3/",
  group_by_length=True,
  per_device_train_batch_size=2,
  gradient_accumulation_steps=16,
  evaluation_strategy="steps",
  num_train_epochs=30,
  #max_steps = 2000,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  load_best_model_at_end = True
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=cv_train,
    eval_dataset=cv_test,
    tokenizer=processor.feature_extractor,
)

In [ ]:
#gc.collect()
#torch.cuda.empty_cache()

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Step,Training Loss,Validation Loss,Cer,Runtime,Samples Per Second
400,46.901000,7.306947,0.999485,69.696600,18.164000
800,7.072800,7.097707,0.999485,63.531200,19.927000


Step,Training Loss,Validation Loss,Cer,Runtime,Samples Per Second
400,46.901000,7.306947,0.999485,69.696600,18.164000
800,7.072800,7.097707,0.999485,63.531200,19.927000
1200,5.935100,4.284671,0.830009,64.794100,19.539000
1600,3.756400,3.522993,0.758847,63.077100,20.071000
2000,3.067000,3.117598,0.717797,64.965800,19.487000
2400,2.551200,2.758144,0.672886,63.277100,20.007000
2800,2.134400,2.492286,0.631064,64.719900,19.561000
3200,1.850700,2.349797,0.607386,63.487000,19.941000
3600,1.615800,2.271907,0.592459,64.170700,19.729000
4000,1.482400,2.198100,0.578819,62.834300,20.148000


TrainOutput(global_step=4170, training_loss=7.383554940715396, metrics={'train_runtime': 18206.1434, 'train_samples_per_second': 0.229, 'total_flos': 1.2290634423088378e+19, 'epoch': 30.0, 'init_mem_cpu_alloc_delta': 900878336, 'init_mem_gpu_alloc_delta': 1273009664, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 4390055936, 'train_mem_gpu_alloc_delta': 5083933696, 'train_mem_cpu_peaked_delta': 2865389568, 'train_mem_gpu_peaked_delta': 3870021632})

In [ ]:
Wav2Vec2ForCTC.save_pretrained(model, save_directory = "/content/drive/MyDrive/585_Team_Folder/model_3/")

In [ ]:
Wav2Vec2Processor.save_pretrained(processor, save_directory = "/content/drive/MyDrive/585_Team_Folder/model_3/")

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained("/content/drive/MyDrive/585_Team_Folder/model_3/").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("/content/drive/MyDrive/585_Team_Folder/model_3/")

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
input_dict = processor(cv_test[0]["input_values"], return_tensors="pt", padding=True)
logits = model(input_dict.input_values.to("cuda")).logits
pred_ids = torch.argmax(logits, dim=-1)[0]

KeyError: ignored

In [ ]:
test_output = test.filter(lambda x: os.path.getsize(x['path']) < 30000)

In [ ]:
print("Prediction:")
print(processor.decode(pred_ids))

Prediction:
二十一一年去士


In [ ]:
print('Reference:')
test_output[0]['sentence']

Reference:


'二十一年去世。'